# Installing libriaries and necessary packages

In [1]:
!pip uninstall torch torchvision torchaudio -y
!pip install transformers ipywidgets gradio --upgrade
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers[torch] accelerate -U
!pip install datasets sacremoses sacrebleu
!pip install huggingface_hub==0.23.0
!pip install accelerate==0.21.0 -U
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U accelerate
!pip install peft
!pip install deepspeed --upgrade

Found existing installation: torch 2.1.1+cu121
Uninstalling torch-2.1.1+cu121:
  Successfully uninstalled torch-2.1.1+cu121
Found existing installation: torchvision 0.16.1+cu121
Uninstalling torchvision-0.16.1+cu121:
  Successfully uninstalled torchvision-0.16.1+cu121
Found existing installation: torchaudio 2.1.1+cu121
Uninstalling torchaudio-2.1.1+cu121:
  Successfully uninstalled torchaudio-2.1.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 87.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━

# Importing necessary libraries

In [2]:
import gradio as gr
import re
from transformers import pipeline, get_scheduler, AdamW, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig, TextStreamer
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_metric
import torch
from torch.utils.data import Dataset
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from peft import get_peft_model, LoraConfig

2024-08-24 14:26:30.090154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 14:26:30.090296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 14:26:30.167230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-24 14:26:30.320266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-24 14:26:31.894427: W tensorflow/compiler/tf2

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Transfert learning and fine tuning the pretrained model

#### Data preprocessing

In [4]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [5]:
df = pd.read_csv('ITIL_20k.csv', encoding='utf-8')
rows, columns = df.shape
print(f"Number of rows: {rows}, Number of columns: {columns}")

Number of rows: 18218, Number of columns: 2


In [6]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [7]:
def normalize_text(text):
    # Convert the text to lowercase
    text = text.lower()
    # Replace multiple spaces with a single space and strip leading/trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove text inside parentheses
    text = re.sub(r'\(.*?\)', '', text)
    # Tokenize the text into words
    words = word_tokenize(text)
    return ' '.join(words)

In [8]:
df['English Text'] = df['English Text'].apply(normalize_text)
df['Arabic Text'] = df['Arabic Text'].apply(normalize_text)
df.head()

,English Text,Arabic Text
0,service quality plan is a plan that describes ...,خطة جودة الخدمة هي خطة تصف النهج لضمان جودة ال...
1,noise nondisclosure agreement nonintrusive mon...,ازعاج اتفاقية عدم الافصاح رصد التطفل المسالم م...
2,hot standby see fast recovery ; immediate reco...,استعداد ساخن مرادف لاسترجاع السرعة أو الاسترجا...
3,asset asset specificity,الأصول مميزات الأصول
4,software as a service,البرمجيات كخدمة


#### Tokenization

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
def tokenize_function(examples):
    return tokenizer(examples['English Text'], text_target=examples['Arabic Text'], truncation=True)

In [11]:
tokenized_data = df.apply(lambda x: tokenize_function({'English Text': x['English Text'], 'Arabic Text': x['Arabic Text']}), axis=1).tolist()

#### Split the data into training and testing set

In [12]:
train_data, eval_data = train_test_split(tokenized_data, test_size=0.2, random_state=42)

In [13]:
class TranslationDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.data[idx].items()}

In [14]:
train_dataset = TranslationDataset(train_data)
eval_dataset = TranslationDataset(eval_data)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

#### Fixing training parameters

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_steps=100,
    eval_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_steps=10,
    fp16=True,
    warmup_steps=500,
    predict_with_generate=True,
    gradient_accumulation_steps=2,
    logging_first_step=True,
    dataloader_num_workers=4
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


#### Optimization function

In [17]:
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
num_train_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Training

In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, lr_scheduler),
)

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [27]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting

Epoch,Training Loss,Validation Loss
1,0.137400,0.066731
2,0.096800,0.053594
3,0.084800,0.046334
4,0.092400,0.041674
5,0.078800,0.040356


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting

{'eval_loss': 0.04035613313317299,
 'eval_runtime': 12.2876,
 'eval_samples_per_second': 296.07,
 'eval_steps_per_second': 9.278,
 'epoch': 5.0}

In [20]:
model.save_pretrained("./en_ar_translator_IFRS")
tokenizer.save_pretrained("./en_ar_translator_tkz_IFRS")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}


('./en_ar_translator_tkz_IFRS/tokenizer_config.json',
 './en_ar_translator_tkz_IFRS/special_tokens_map.json',
 './en_ar_translator_tkz_IFRS/vocab.json',
 './en_ar_translator_tkz_IFRS/source.spm',
 './en_ar_translator_tkz_IFRS/target.spm',
 './en_ar_translator_tkz_IFRS/added_tokens.json')

In [21]:
fine_tuned_pipe = pipeline("translation", model="./en_ar_translator_IFRS", tokenizer="./en_ar_translator_tkz_IFRS")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [22]:
def translate_tuned(text):
    results = fine_tuned_pipe(text.lower())
    return results[0]['translation_text']

interface = gr.Interface(
    fn=translate_tuned,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here..."),
    outputs=gr.Textbox(lines=5),
    title="English to Arabic Translator for IFRS Terminology")
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b9268e86b23aceab70.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Testing the model

In [23]:
data = [
    ("a project code", "رمز المشروع"),
    ("a statement of cash flows for the period", "قائمة التدفقات النقدية للفترة"),
    ("a statement of changes in equity for the period", "قائمة التغيرات في حقوق الملكية للفترة"),
    ("a statement of financial position as at the end of the period", "قائمة المركز المالي كما في نهاية الفترة"),
    ("a statement of profit or loss and other comprehensive income for the period", "قائمة الربح أو الخسارة والدخل الشامل الاخر للفترة"),
    ("Abbreviated Accounts", "‫حسابات مختصرة‬"),
    ("Above-The-Line", "‫فوق الخط‬"),
    ("Abridged Accounts", "‫حسابات موجزة‬"),
    ("Accelerated Depreciation", "‫استھلاك متسارع‬"),
    ("Acceptances Credit", "‫ائتمان القبول‬"),
    ("Accord And Satisfaction", "‫التوافق والرضا‬"),
    ("Account Analysis", "‫تحلیل الحساب‬"),
    ("Account Classification Method", "‫طریقة تصنیف الحساب‬‏"),
    ("Accountability", "‫مسئولیة‬"),
    ("Accountancy", "‫المحاسبة‬"),
    ("Accountancy Bodies", "‫ھیئات محاسبة‬"),
    ("Accountancy Investigation And Disciplinary Board", "‫مجلس التحقیق والتأدیب المحاسبي"),
    ("Accountant", "‫محاسب‬"),
    ("Accounting", "‫محاسبة‬"),
    ("Accounting And Auditing Organization For Islamic Financial Institutions", "‫ھیئة المحاسبة والمراجعة للمؤسسات المالیة الإسلامية"),
    ("Accounting And Finance Association Of Australia And New Zealand", "‫جمعیة المحاسبة والمالیة في أسترالیا ونیوزیلندا"),
    ("Accounting Assumptions", "‫الافتراضات المحاسبیة‬"),
    ("Accounting Basis", "‫الأساس المحاسبي‬"),
    ("Accounting Concepts", "‫المفاھیم المحاسبة‬"),
    ("Accounting Cushion", "‫وسادة المحاسبة‬"),
    ("Accounting Cycle", "‫الدائرة المحاسبیة‬"),
    ("Accounting Entity", "‫الوحدة المحاسبیة‬"),
    ("Accounting Equation", "‫معادلة محاسبیة‬"),
    ("Accounting Estimates", "‫التقدیرات المحاسبیة‬"),
    ("Accounting Manual", "‫دلیل المحاسبة‬"),
    ("Accounting Period", "‫فترة المحاسبة‬"),
    ("Accounting Plan", "‫خطة المحاسبة‬"),
    ("Accounting policies", "السياسات المحاسبية"),
    ("Accounting Policies", "‫السیاسات المحاسبة‬"),
    ("Accounting Policy", "سياسة محاسبية"),
    ("Accounting Principles", "‫المبادئ المحاسبة‬"),
    ("Accounting Principles Board", "‫مجلس مبادئ المحاسبة"),
    ("Accounting Profit", "‫الربح المحاسبي‬"),
    ("Accounting Rate Of Return", "‫معدل العائد المحاسبي"),
    ("Accounting Records", "‫السجلات المحاسبیة‬"),
    ("Accounting Regulatory Committee", "‫لجنة تنظیم المحاسبة ‬"),
    ("Accounting Standards", "‫المعاییر المحاسبیة‬"),
    ("Accounting Standards Board", "‫مجلس معاییر المحاسبة"),
    ("Accounting Standards Board Of Japan", "‫مجلس معاییر المحاسبة الیاباني"),
    ("Accounting treatment", "المعالجة المحاسبية"),
    ("Accounts", "‫حسابات‬"),
    ("Accounts Payable", "حسابات الذمم الدائنة"),
    ("Accounts Payable Aging", "تقادم الحسابات الدائنة"),
    ("accounts receivable", "حسابات الذمم المدينة"),
]

testset = pd.DataFrame(data, columns=['English Text', 'Arabic Text'])

In [24]:
translated_texts = []
for text in testset['English Text']:
    translation = fine_tuned_pipe(text, max_length=400)
    translated_texts.append(translation[0]['translation_text'])

In [25]:
testset['Translated Text'] = translated_texts

In [26]:
testset

,English Text,Arabic Text,Translated Text
0,a project code,رمز المشروع,شفرة المشروع
1,a statement of cash flows for the period,قائمة التدفقات النقدية للفترة,هو بيان بالتدفقات النقدية لتلك الفترة
2,a statement of changes in equity for the period,قائمة التغيرات في حقوق الملكية للفترة,بيان التغييرات في الأسهم للفترة
3,a statement of financial position as at the en...,قائمة المركز المالي كما في نهاية الفترة,هو بيان الوضع المالي في نهاية الفترة
4,a statement of profit or loss and other compre...,قائمة الربح أو الخسارة والدخل الشامل الاخر للفترة,بيان الربح أو الخسارة وغير ذلك من الإيرادات ال...
5,Abbreviated Accounts,‫حسابات مختصرة‬,‫المحاسبة المفردة‬
6,Above-The-Line,‫فوق الخط‬,‫فوق الخط‬
7,Abridged Accounts,‫حسابات موجزة‬,‫المحاسبة المكهنة‬
8,Accelerated Depreciation,‫استھلاك متسارع‬,‫الاستهلاك المعجل‬
9,Acceptances Credit,‫ائتمان القبول‬,القبول الاعتماد


### Accuracy = 1 - 3/50 = 94%